In [1]:
using Plots
using StatsBase
using LinearAlgebra
using Statistics
using JLD2
using Dates
using LsqFit
using StaticArrays
using JSON

# data structure module (needs to be module to avoid overwriting local variables in REPL scope)
include("SpinSimParams.jl")
using .SpinSimParams

# function libraries
include("spin_sims.jl");

# Set Independent Variables

In [5]:
# interaction
α = collect(LinRange(0.1, 0.2, 10));
ω = 0;

# flip operators
Ix = @SMatrix [0 1/2; 1/2 0];
f = LinRange(0, 1, 10);
UL90 = ();
UR90 = ();
for idx = 1:10
    UL90 = (UL90..., exp(-1im*pi*f[idx]*Ix/2));
    UR90 = (UR90..., exp(1im*pi*f[idx]*Ix/2));
end

# number of frequencies
nx = 20;
ny = 20;
nz = 1;
n = (nx, ny, nz);
nfreq = prod(n);

# make the parameter file
params = make_params(α, ω, n);
    
params["UL90"] = UL90;
params["UR90"] = UR90;
params["dt"] = 1;

# Set Options
1. "integrated_sampling": if true, use integrated sampling; if false, use discrete sampling (default = false).
2. "tiles": if true, use tile approach to generating lattice (default = false).  Must provide **params["r_tiles"] and params["tile_norm"]**.  "r_tiles" specifies the positions of the tiles, and "tile_norm" is a rescaling parameter to make sure the tiles do not overlap.
3. "local_M_on": if true, implement correlation length (default = false).  Must provide **params["M_stencil"]**.
4. $\theta$ is defined as follows: $\theta$[1] is the angle between hlk[1] and hlk[2], $\theta$[2] is the angle between hlk[1] and the projection of hlk[3] into the plan of hlk[1] and hlk[2], and $\theta$[3] is the angle between hlk[3] and the axis perpendicular to the plan created by hlk[1] and hlk[2] (ie if hlk[1] and hlk[2] are considered in the x-y plane, $\theta$[3] is the angle from the z axis to hlk[3])

In [6]:
# set the variables to loop over
params["integrated_sampling"] = true;
vars = ["α", ("UL90", "UR90")];

In [ ]:
I, d = make_idx(vars, params)
M_list = Array{Any}(undef, d)

for i in I
    
    # generate temporary parameters
    tparams = make_temp_params(params, vars, i)

    # simulate
    @time M_list[i] = spin_echo_sim(tparams)
    
end

@save "002_M_list.jld2" M_list;

using JSON
fname = "002_parameters.json"
open(fname,"w") do f
    JSON.print(f, params)
end